In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
max_features = 40000
maxlen = 200

def clean_text( text ):
    #https://www.kaggle.com/sreeram004/test-lr-with-convai-dataset
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(95851, 6)


In [2]:
list_sentences_train[:5]

array([ 'nonsense kiss off geek what i said is true i will have your account terminated ',
       ' please do not vandalize pages as you did with this edit to w s merwin if you continue to do so you will be blocked from editing ',
       ' points of interest i removed the points of interest section you added because it seemed kind of spammy i know you probably did not mean to disobey the rules but generally a point of interest tends to be rather touristy and quite irrelevant to an area culture that just my opinion though if you want to reply just put your reply here and add talkback jamiegraham08 on my talkpage ',
       'asking some his nationality is a racial offence wow was not aware of it blocking me has shown your support towards your community thanku for that',
       'the reader here is not going by my say so for ethereal vocal style and dark lyrical content the cited sources in the external links are saying those things if you feel the sources are unreliable or i did not repres

In [3]:
print('test len',len(test))

test len 226998


In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(95851, 200) (226998, 200)


In [5]:
# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

soooooooooon 97830
paa 78647
walleradded 125213
jmundo 57469
reorgnization 115312
272001018 99980


In [6]:
# read word2vec
# https://github.com/facebookresearch/MUSE
word_vec_dict = {}

with open('../wiki.multi.fr.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

with open('../wiki.multi.de.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

with open('../wiki.multi.en.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
        
print(len(word_vec_dict))


435713


In [7]:
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

# pre null 5293

del word_vec_dict

Preparing embedding matrix
Null word embeddings: 4716


In [10]:
from sklearn.metrics import log_loss,accuracy_score
from keras.optimizers import Adam

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_lstm_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.01),
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [11]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_lstm_model()
        batch_size = 64
        epochs = 12
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train(3,3)
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/lstm_muse_1_csv_adj3_add_de_fr.gz", index=False, compression='gzip')
import pickle
with open('../features/lstm_muse_adj3_add_de_fr_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')


Train on 63900 samples, validate on 31951 samples
Epoch 1/12
63900/63900 [==============================] - 770s 12ms/step - loss: 0.0624 - acc: 0.9778 - val_loss: 0.0480 - val_acc: 0.9818
Epoch 2/12
63900/63900 [==============================] - 764s 12ms/step - loss: 0.0499 - acc: 0.9817 - val_loss: 0.0460 - val_acc: 0.9827
Epoch 3/12
63900/63900 [==============================] - 765s 12ms/step - loss: 0.0459 - acc: 0.9825 - val_loss: 0.0454 - val_acc: 0.9830
Epoch 4/12
63900/63900 [==============================] - 765s 12ms/step - loss: 0.0443 - acc: 0.9832 - val_loss: 0.0489 - val_acc: 0.9819
Epoch 5/12
63900/63900 [==============================] - 768s 12ms/step - loss: 0.0429 - acc: 0.9836 - val_loss: 0.0450 - val_acc: 0.9829
Epoch 6/12
63900/63900 [==============================] - 760s 12ms/step - loss: 0.0417 - acc: 0.9838 - val_loss: 0.0457 - val_acc: 0.9828
Epoch 7/12
63900/63900 [==============================] - 765s 12ms/step - loss: 0.0399 - acc: 0.9843 - val_loss: 0.

In [12]:
train_pred[:10].round(3)

array([[ 0.493,  0.   ,  0.115,  0.   ,  0.129,  0.001],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   ,  0.001,  0.   ,  0.   ,  0.   ],
       [ 0.003,  0.   ,  0.002,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.121,  0.001,  0.027,  0.003,  0.034,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.189,  0.001,  0.008,  0.035,  0.022,  0.003],
       [ 0.022,  0.   ,  0.005,  0.   ,  0.002,  0.004],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [13]:
y[:10]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])